In [1]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import import_gemini
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload
import other_source_data

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(import_gemini)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)
importlib.reload(other_source_data)

from import_default import *
from import_database import *
from import_other import *
from import_gemini import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *
from other_source_data import *

#### Dữ liệu từ Mongo

In [2]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification_df = get_mongo_collection(ref_db, 'current_quarter_classification')

if date_series['date'].max() < get_today_date():
    new_date = get_today_date()
    date_series = pd.concat([date_series, pd.DataFrame({'date': [new_date]})]).sort_values(by='date', ascending=False).reset_index(drop=True)

In [3]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_prev_high": 1,
    "week_prev_low": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
    'WPIVOT_P': 1,
    'MPIVOT_P': 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [4]:
today_group_df = get_mongo_collection(stock_db, "today_group")
history_group_df = get_mongo_collection(stock_db, "history_group")
full_group_df = pd.concat([today_group_df, history_group_df], axis=0, ignore_index=True)

In [5]:
itd_index_df = get_mongo_collection(stock_db, "itd_index")

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

#### Tạo các models cho gemini

In [6]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

#### Dữ liệu thống kê giao dịch phiên hôm trước

- Tính toán bảng dữ liệu

In [7]:
itd_vnindex_df = itd_index_df[itd_index_df['ticker'] == 'VNINDEX'].reset_index(drop=True)
eod_vnindex_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].reset_index(drop=True)
vnindex_close_prev = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['close'].item()

itd_vnindex_df['pct_change'] = (itd_vnindex_df['close'] - vnindex_close_prev) / vnindex_close_prev
itd_vnindex_df['diff'] = itd_vnindex_df['close'] - vnindex_close_prev
itd_vnindex_df['cum_volume'] = itd_vnindex_df['volume'][::-1].cumsum()[::-1]
itd_vnindex_df['cum_value'] = eod_vnindex_df['option'].iloc[0]

itd_vnindex_df['week_open'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['week_open'].item()
itd_vnindex_df['week_prev_high'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['week_prev_high'].item()
itd_vnindex_df['week_prev_low'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['week_prev_low'].item()
itd_vnindex_df['week_pivot'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['WPIVOT_P'].item()

itd_vnindex_df['month_open'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['month_open'].item()
itd_vnindex_df['month_prev_high'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['month_prev_high'].item()
itd_vnindex_df['month_prev_low'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['month_prev_low'].item()
itd_vnindex_df['month_pivot'] = eod_vnindex_df[eod_vnindex_df['date'] < itd_vnindex_df['date'].iloc[-1].normalize()].iloc[0]['MPIVOT_P'].item()

daily_9h00_itd_df = itd_vnindex_df.sort_values('date').copy()

- Viêt nhận xét ngắn

In [8]:
def daily_vnindex_overall_comment(df):
    prompt = f"""
    Đây là dữ liệu của chỉ số VNINDEX:
    {df.to_csv(index=False, sep='|', lineterminator='\n')}

    **Bối cảnh:**
    Hãy đóng vai là một nhà phân tích tài chính chuyên nghiệp và khách quan của một công ty chứng khoán. Nhiệm vụ của bạn là phân tích dữ liệu giá theo phút của hợp đồng tương lai (ticker vnindex) được cung cấp và viết một báo cáo ngắn gọn, súc tích về diễn biến của phiên giao dịch đó.

    **Dữ liệu đầu vào:**
    Dữ liệu là một chuỗi văn bản chứa thông tin giao dịch theo từng phút, bao gồm các cột: `date`, `ticker`, `open`, `high`, `low`, `close`.

    **Yêu cầu đầu ra:**
    Kết quả trả về phải tuân thủ nghiêm ngặt các quy tắc sau:

    * **Định dạng:** Chỉ trả về **duy nhất** đoạn văn phân tích. **Tuyệt đối không** thêm bất kỳ câu dẫn dắt, tiêu đề, hay lời chào nào (ví dụ: không được viết "Dưới đây là báo cáo..." hoặc "Sau khi phân tích...").
    * **Cấu trúc:** Đoạn văn phải có chính xác **3 câu**. Mỗi câu từ 25 đến 30 từ.
    * **Giọng điệu:** Chuyên nghiệp, khách quan, mang tính tường thuật, không đưa ra khuyến nghị mua/bán.
    * **Nguồn:** Mọi phân tích phải dựa hoàn toàn vào dữ liệu được cung cấp.
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.

    **Chi tiết hướng dẫn cho từng câu:**

    * **Câu 1: Bối cảnh và Diễn biến mở đầu (khoảng 15-20 từ)**
        * **Mục đích:** Giới thiệu bối cảnh phiên giao dịch và mô tả xu hướng ngay sau khi thị trường mở cửa.
        * **Hướng dẫn nội dung:**
            1.  Bắt đầu câu bằng cụm từ "Trong phiên giao dịch ngày `[DD/MM/YYYY]`...". Bạn phải tự động trích xuất và định dạng ngày từ cột `date` trong dữ liệu.
            2.  Nhận xét ngắn gọn về trạng thái mở cửa (ví dụ: "mở cửa khá ổn định", "mở cửa trong sắc xanh", "chịu áp lực ngay từ đầu").
            3.  Mô tả hành động giá ngay sau đó (ví dụ: "nhưng nhanh chóng chịu áp lực bán", "tuy nhiên đà tăng không duy trì được", "và tiếp tục nới rộng đà giảm").

    * **Câu 2: Diễn biến chính và Mức biến động (khoảng 15-20 từ)**
        * **Mục đích:** Tóm tắt xu hướng chủ đạo trong phiên và định lượng mức độ biến động mạnh nhất.
        * **Hướng dẫn nội dung:**
            1.  Mô tả xu hướng chính kéo dài trong phiên (ví dụ: "Xu hướng tiêu cực này chiếm ưu thế", "Lực mua chủ động xuyên suốt phiên").
            2.  Làm nổi bật mức biến động lớn nhất bằng cách tính chênh lệch giữa giá trị `high` cao nhất và `low` thấp nhất trong toàn bộ dữ liệu. Diễn đạt nó dưới dạng "đẩy chỉ số có thời điểm giảm hơn `X` điểm so với mức cao nhất" hoặc "giúp chỉ số có lúc tăng hơn `X` điểm so với mức điểm thấp nhất" .

    * **Câu 3: Tổng kết cuối phiên (khoảng 15-20 từ)**
        * **Mục đích:** Mô tả các nỗ lực vào cuối phiên và chốt lại trạng thái của giá đóng cửa.
        * **Hướng dẫn nội dung:**
            1.  Nhận xét về các diễn biến trong giai đoạn cuối phiên (ví dụ: "Mặc dù một nỗ lực phục hồi...", "Áp lực bán gia tăng trở lại...", "Xu hướng vẫn tiếp diễn...").
            2.  Đưa ra kết luận về giá đóng cửa so với mặt bằng chung của phiên (ví dụ: "chỉ số vẫn duy trì ở vùng giá thấp", "bảo toàn được thành quả", "thu hẹp đáng kể đà giảm").
    """
    return prompt

overall_vnindex_comment = generate_content_with_model_dict(standard_model_dict, daily_vnindex_overall_comment(itd_vnindex_df), 'daily_vnindex_overall_comment')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vnindex_overall_comment.


#### Dòng tiền và biến động tương quan

In [9]:
temp_group_df = date_series.iloc[:20].set_index('date')
temp_group_df['pct_change'] = full_index_df[full_index_df['ticker'] == 'VNINDEX'].iloc[:20].set_index('date')['pct_change']
temp_df = full_group_df[full_group_df['ticker'] == 'all'].set_index('date')
temp_group_df['vol_ratio'] = temp_df['vol_ratio']
temp_group_df['t0_score'] = temp_df['t0_score']
temp_group_df['t5_score'] = temp_df['t5_score']

temp_group_df['pct_score'] = temp_group_df['t0_score'][::-1].cumsum()[::-1]/100
temp_group_df['pct_score'].iloc[-1] = 0
temp_group_df['pct_price'] = temp_group_df['pct_change'][::-1].cumsum()[::-1]
temp_group_df['pct_price'].iloc[-1] = 0

daily_9h00_flow_df = temp_group_df.reset_index()

#### Dữ liệu nước ngoài và tự doanh

In [10]:
nntd_data_list = []
for type_name in ['NN', 'TD']:
    temp_df = nntd_index_df[(nntd_index_df['ticker'].isin(['HNXINDEX', 'UPINDEX', 'VNINDEX'])) & (nntd_index_df['type'] == type_name)].reset_index(drop=True)
    temp_df = temp_df.groupby(['date']).sum().iloc[:, :6].reset_index().sort_values(by='date', ascending=False)
    temp_df = temp_df[temp_df['net_value'] != 0].reset_index(drop=True)
    temp_df['type'] = type_name
    nntd_data_list.append(temp_df)

daily_9h00_nntd_data_df = pd.concat(nntd_data_list, ignore_index=True)

In [11]:
nntd_stock_list = []
for type_name in ['NN', 'TD']:
    type_df = pd.DataFrame()
    for ticker in nntd_stock_df['ticker'].unique():
        temp_df = nntd_stock_df[(nntd_stock_df['ticker'] == ticker) & (nntd_stock_df['type'] == type_name)]
        temp_df = temp_df.sort_values('date', ascending=False).reset_index(drop=True)
        temp_df = temp_df[temp_df['net_value'] != 0].reset_index(drop=True)
        if len(temp_df) > 0:
            type_df = pd.concat([type_df, temp_df[['type', 'ticker', 'net_value']].iloc[[0]]], ignore_index=True).sort_values('net_value', ascending=False)

    # Gán kết quả và tổng hợp
    result_df = pd.concat([type_df.head(10), type_df.tail(10)], ignore_index=True)
    result_df['top_check'] = (result_df['net_value'] > 0).astype(int)
    result_df['net_value'] = abs(result_df['net_value'])
    result_df['type'] = type_name
    nntd_stock_list.append(result_df)

daily_9h00_nntd_stock_df = pd.concat(nntd_stock_list, ignore_index=True)

#### Phân tích kĩ thuật VNINDEX

In [12]:
vnindex_chart_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].iloc[:120].sort_values('date').reset_index(drop=True)
image_name = 'TA_DAILY_VNINDEX.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            vnindex_chart_df, # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            path='../output/cts_weekly_chart/',
            symbol_name='VNINDEX',
            image_name = image_name
        )
        upload_to_r2(fig_images, image_name)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()

In [13]:
def daily_vnindex_ta_analysis(itd_df, eod_df):
    prompt = f"""
Đây là dữ liệu trong phiên gần nhất của chỉ số VNINDEX:
    {itd_df[['date', 'ticker', 'open', 'high', 'low', 'close']].to_csv(index=False, sep='|', lineterminator='\\n')}

Đây là dữ liệu lịch sử của chỉ số hợp đồng tương lai VNINDEX:
    {eod_df.to_csv(index=False, sep='|', lineterminator='\\n')}

**Vai trò:** Bạn là một **nhà phân tích kỹ thuật cấp cao**, chuyên về phái sinh, có nhiệm vụ đưa ra nhận định kỹ thuật khách quan và sắc bén.

**Nhiệm vụ:** Dựa trên dữ liệu và các chỉ báo kỹ thuật, hãy viết một báo cáo phân tích về trạng thái kỹ thuật của chỉ số cho **phiên kế tiếp**, tuân thủ nghiêm ngặt cấu trúc **8 câu, 3 đoạn**.

**Yêu cầu về nội dung, cấu trúc và định dạng:**
    Báo cáo phải gồm **8 câu, mỗi câu dài khoảng 20 đến 25 từ** và được chia thành **3 đoạn văn riêng biệt**. Bắt đầu thẳng vào nội dung, không có lời chào.

    **Đoạn 1 (3 câu): Phân tích Phiên Gần Nhất dựa vào dự liệu trong phiên gần nhất**
        * **Câu 1:** Viết một câu duy nhất mô tả phiên gần nhất theo đúng cấu trúc sau: **"Trong phiên (dd/mm), chỉ số [Trạng thái mở cửa so với phiên trước] và [Diễn biến ngắn gọn đầu và cuối phiên]"** đã hình thành nên một **"[Mô tả hình thái nến chi tiết]"**, thể hiện **"[Kết luận về phe thắng thế hoặc sự giằng co trong phiên]"**.
        * **Câu 2:** Phân tích vai trò của hai đường **SMA 20 và SMA 60** trong việc xác nhận xu hướng ngắn hạn và trung hạn hiện tại.
        * **Câu 3:** Đánh giá chỉ báo **RSI 14**, nêu bật trạng thái hiện tại của chỉ báo (ví dụ: vùng trung tính, quá mua/bán) và ý nghĩa của nó.

    **Đoạn 2 (3 câu): Xác định các Vùng giá Cốt lõi dựa vào giữ liệu lịch sử**
        * **Câu 4:** Phân tích và xác định mốc hỗ trợ/kháng cự quan trọng theo **khung Tuần** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 5:** Phân tích và xác định mốc hỗ trợ/kháng cự quan trọng theo **khung Tháng** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 6:** Đưa ra kết luận và nhấn mạnh **một vùng hỗ trợ và một vùng kháng cự then chốt nhất** sẽ quyết định xu hướng trong các phiên tới.

    **Đoạn 3 (2 câu): Kết luận Kỹ thuật**
        * **Câu 7:** Đưa ra một câu nhận định tổng thể, khẳng định xu hướng kỹ thuật chủ đạo hiện tại của chỉ số (tăng, giảm hoặc đi ngang).
        * **Câu 8:** Ngay sau đó, chỉ ra các vùng giá hỗ trợ và kháng cự then chốt và giải thích tại sao (ý là các vùng này là của chỉ báo nào và sức mạnh thế nào).

**Yêu cầu về ngôn ngữ và trình bày:**
    * **Văn phong:** Sắc bén, khách quan, quyết đoán và đa dạng, tránh lặp lại cấu trúc câu.
    * **TUYỆT ĐỐI KHÔNG** sử dụng dấu ;.
    * **TUYỆT ĐỐI KHÔNG** được viết tên cột dữ liệu gốc và **KHÔNG** có lời chào hay câu dẫn dắt.
    * **TUYỆT ĐỐI KHÔNG** sử dụng các từ nối mang tính tổng kết ở đầu câu, cứ đi thẳng vào nội dung chính (ví dụ: "Nhìn chung", "Do đó", "Theo đó", "Tổng thể").
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.
    """
    return prompt

ta_vnindex_comment = generate_content_with_model_dict(standard_model_dict, daily_vnindex_ta_analysis(itd_vnindex_df, vnindex_chart_df), 'daily_vnindex_ta_analysis')


✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vnindex_ta_analysis.


#### Lưu vào MSSQL

In [14]:
daily_9h00_comment_df = pd.DataFrame(
    {
        "type":     ["overall", "ta_vnindex", "date", "time"],
        "comment": [
            overall_vnindex_comment,
            ta_vnindex_comment,
            time_series.iloc[0].item().strftime('%d/%m/%Y'),
            time_series.iloc[0].item().strftime('%H:%M')
        ],
    }
)

In [15]:
%%capture
save_to_mssql(cts_engine, daily_9h00_comment_df, 'daily_9h00_comment')
save_to_mssql(cts_engine, daily_9h00_itd_df, 'daily_9h00_itd')
save_to_mssql(cts_engine, daily_9h00_nntd_data_df, 'daily_9h00_nntd_data')
save_to_mssql(cts_engine, daily_9h00_nntd_stock_df, 'daily_9h00_nntd_stock')
save_to_mssql(cts_engine, daily_9h00_flow_df, 'daily_9h00_flow')